<a href="https://colab.research.google.com/github/MDRobiulhassan/Machine-Learning-Course/blob/main/College_Placement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, log_loss

In [2]:
df=pd.read_csv('/content/drive/MyDrive/University/7th Semester/MLL/colab/college_student_placement_dataset.csv')

In [3]:
df.head()

,College_ID,IQ,Prev_Sem_Result,CGPA,Academic_Performance,Internship_Experience,Extra_Curricular_Score,Communication_Skills,Projects_Completed,Placement
0,CLG0030,107,6.61,6.28,8,No,8,8,4,No
1,CLG0061,97,5.52,5.37,8,No,7,8,0,No
2,CLG0036,109,5.36,5.83,9,No,3,1,1,No
3,CLG0055,122,5.47,5.75,6,Yes,1,6,1,No
4,CLG0004,96,7.91,7.69,7,No,8,10,2,No


In [4]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   College_ID              10000 non-null  object 
 1   IQ                      10000 non-null  int64  
 2   Prev_Sem_Result         10000 non-null  float64
 3   CGPA                    10000 non-null  float64
 4   Academic_Performance    10000 non-null  int64  
 5   Internship_Experience   10000 non-null  object 
 6   Extra_Curricular_Score  10000 non-null  int64  
 7   Communication_Skills    10000 non-null  int64  
 8   Projects_Completed      10000 non-null  int64  
 9   Placement               10000 non-null  object 
dtypes: float64(2), int64(5), object(3)
memory usage: 781.4+ KB
None


In [5]:
df.isnull().sum()

,0
College_ID,0
IQ,0
Prev_Sem_Result,0
CGPA,0
Academic_Performance,0
Internship_Experience,0
Extra_Curricular_Score,0
Communication_Skills,0
Projects_Completed,0
Placement,0


In [6]:
df.duplicated().sum()

np.int64(0)

In [7]:
df['Placement'] = df['Placement'].map({'No': 0, 'Yes': 1})
df['Internship_Experience'] = df['Internship_Experience'].map({'No': 0, 'Yes': 1})

In [8]:
X = df[['IQ', 'Prev_Sem_Result', 'CGPA', 'Academic_Performance',
        'Internship_Experience', 'Extra_Curricular_Score',
        'Communication_Skills', 'Projects_Completed']]
y = df['Placement']


In [9]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [10]:
from sklearn.preprocessing import PolynomialFeatures

# Create polynomial features (degree 2 is a good start)
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X_scaled)

print("Original feature count:", X_scaled.shape[1])
print("Polynomial feature count:", X_poly.shape[1])

Original feature count: 8
Polynomial feature count: 44


In [11]:
feature_names = poly.get_feature_names_out(X.columns)
print(feature_names)

['IQ' 'Prev_Sem_Result' 'CGPA' 'Academic_Performance'
 'Internship_Experience' 'Extra_Curricular_Score' 'Communication_Skills'
 'Projects_Completed' 'IQ^2' 'IQ Prev_Sem_Result' 'IQ CGPA'
 'IQ Academic_Performance' 'IQ Internship_Experience'
 'IQ Extra_Curricular_Score' 'IQ Communication_Skills'
 'IQ Projects_Completed' 'Prev_Sem_Result^2' 'Prev_Sem_Result CGPA'
 'Prev_Sem_Result Academic_Performance'
 'Prev_Sem_Result Internship_Experience'
 'Prev_Sem_Result Extra_Curricular_Score'
 'Prev_Sem_Result Communication_Skills'
 'Prev_Sem_Result Projects_Completed' 'CGPA^2' 'CGPA Academic_Performance'
 'CGPA Internship_Experience' 'CGPA Extra_Curricular_Score'
 'CGPA Communication_Skills' 'CGPA Projects_Completed'
 'Academic_Performance^2' 'Academic_Performance Internship_Experience'
 'Academic_Performance Extra_Curricular_Score'
 'Academic_Performance Communication_Skills'
 'Academic_Performance Projects_Completed' 'Internship_Experience^2'
 'Internship_Experience Extra_Curricular_Score'
 'I

In [12]:
from sklearn.model_selection import train_test_split

X = df.drop('Placement', axis=1)
y = df['Placement']

# Assuming we already scaled X if needed
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

In [13]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)